In [1]:
from pydrake.all import (SceneGraphCollisionChecker, 
                         Rgba, 
                         Sphere, 
                         RigidTransform, 
                         RotationMatrix,
                         VisibilityGraph)
from iris_environments.environments import plant_builder_2dof_triangle_with_hole
import ipywidgets as widgets
from functools import partial
import numpy as np

plant, scene_graph, diagram, diagram_context, plant_context,meshcat = plant_builder_2dof_triangle_with_hole(True)
# meshcat.SetProperty('/drake/proximity','visible', True)
scene_graph_context = scene_graph.GetMyMutableContextFromRoot(
    diagram_context)
robot_instances = ['cspace_robot']
checker = SceneGraphCollisionChecker(model = diagram, 
                                     robot_model_instances = [plant.GetModelInstanceByName(r) for r in robot_instances], 
                                     edge_step_size = 0.1)

INFO:drake:Meshcat listening for connections at http://localhost:7001
INFO:drake:Allocating contexts to support implicit context parallelism 20


http://localhost:7001


In [2]:
q = np.zeros(plant.num_positions()) 
sliders = []
for i in range(plant.num_positions()):
    q_low = plant.GetPositionLowerLimits()[i]*0.99
    q_high = plant.GetPositionUpperLimits()[i]*0.99
    sliders.append(widgets.FloatSlider(min=q_low, max=q_high, value=0, step=0.001, description=f"q{i}"))

col_col =  Rgba(0.8, 0.0, 0, 0.5)    
col_free =  Rgba(0.0, 0.8, 0.5, 0.5) 
def showres(qvis):
    plant.SetPositions(plant_context, qvis)
    diagram.ForcedPublish(diagram_context)
    query = plant.get_geometry_query_input_port().Eval(plant_context)
    col = query.HasCollisions()
    if col:
        meshcat.SetObject(f"/drake/visualizer/shunk",
                                   Sphere(0.2),
                                   col_col)
    else:
        meshcat.SetObject(f"/drake/visualizer/shunk",
                                   Sphere(0.2),
                                   col_free)
    meshcat.SetTransform(f"/drake/visualizer/shunk",
                                   RigidTransform(RotationMatrix(),
                                                  np.array([0,0,2])))
    return col

def handle_slider_change(change, idx):
    q[idx] = change['new']
    showres(q)
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

FloatSlider(value=0.0, description='q0', max=5.9399999999999995, min=-5.9399999999999995, step=0.001)

FloatSlider(value=0.0, description='q1', max=5.9399999999999995, min=-5.9399999999999995, step=0.001)

In [3]:
from cspace_utils.environment_helpers import get_sample_cfree_handle
def col_free(q):
    return not checker.CheckConfigCollisionFree(q)

cfree_sampler = get_sample_cfree_handle(plant.GetPositionLowerLimits(), 
                                plant.GetPositionUpperLimits(),
                                col_free)

In [4]:
N = 200
samps,_ = cfree_sampler(N, 1000,[])
adj = VisibilityGraph(checker, samps.T, parallelize = True)


In [5]:
from cspace_utils.graphs import double_greedy_clique_cover

clique_cover = double_greedy_clique_cover(adj, min_clique_size=1)

[CCDGDRAKE] remaining nodes 200
[CCDGDRAKE] remaining nodes 77
[CCDGDRAKE] remaining nodes 40
[CCDGDRAKE] remaining nodes 16
[CCDGDRAKE] remaining nodes 2


In [6]:
from cspace_utils.plotting import (plot_points, 
                                   plot_vpoly_2d_meshcat, 
                                   plot_edges_clique,
                                   plot_visibility_graph)
from cspace_utils.colors import generate_maximally_different_colors

nodes_for_plot = np.concatenate((samps, np.zeros((samps.shape[0],1))), axis = 1)
nodes_for_plot = nodes_for_plot[:, [0,2,1]] 

colors = generate_maximally_different_colors(len(clique_cover))
colors =[Rgba(c[0], c[1],c[2],1) for c in colors]
for idx, clique in enumerate(clique_cover):
    col = colors[idx]
    plot_edges_clique(meshcat, clique, nodes_for_plot, f'cl_{idx}', col)

# plot_points(meshcat, nodes_for_plot, 'nodes', size=0.05, color = Rgba(0,0,0,1))
# plot_visibility_graph(meshcat, nodes_for_plot, adj, 'vg')

In [25]:
colors

[(0.8, 0.07999999999999999, 0.07999999999999999),
 (0.07999999999999999, 0.8, 0.07999999999999999),
 (0.07999999999999999, 0.07999999999999999, 0.8)]